In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply preprocessing & augmentation
datagen = ImageDataGenerator(
    rescale=1./255,   # normalize pixels
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    validation_split=0.2   # split training data into train/val
)

# Training generator
train_gen = datagen.flow_from_directory(
    r"C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Train_data",
    target_size=(224,224),   # resize all images
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

# Validation generator
val_gen = datagen.flow_from_directory(
    r"C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Train_data",
    target_size=(224,224),   # resize all images
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

# Test generator (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    r"C:\Users\moham\Downloads\Robotech\Summer_Training\Project\Note_Book\Test_data",
    target_size=(224,224),   # resize all images
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

print("Classes:", train_gen.class_indices)


Found 19440 images belonging to 27 classes.
Found 4860 images belonging to 27 classes.
Found 2700 images belonging to 27 classes.
Classes: {'A': 0, 'B': 1, 'Blank': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26}


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Base model
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))

# Freeze the entire base model first
for layer in base_model.layers:
    layer.trainable = False

# Add custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(train_gen.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Compile for head training
model.compile(optimizer=Adam(learning_rate=1e-3),   # slightly higher LR for head
              loss="categorical_crossentropy",
              metrics=["accuracy"])

with tf.device("/GPU:0"):
    history_stage1 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10,
        callbacks=[early_stop],
        batch_size=64
    )


Epoch 1/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 612s 1s/step - accuracy: 0.6842 - loss: 1.0382 - val_accuracy: 0.7759 - val_loss: 0.7561
Epoch 2/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 582s 958ms/step - accuracy: 0.7439 - loss: 0.8247 - val_accuracy: 0.8016 - val_loss: 0.6661
Epoch 3/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 592s 974ms/step - accuracy: 0.7600 - loss: 0.7536 - val_accuracy: 0.8144 - val_loss: 0.6232
Epoch 4/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 619s 1s/step - accuracy: 0.7785 - loss: 0.7117 - val_accuracy: 0.8146 - val_loss: 0.6106
Epoch 5/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 580s 954ms/step - accuracy: 0.7807 - loss: 0.6919 - val_accuracy: 0.8117 - val_loss: 0.6125
Epoch 6/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 584s 960ms/step - accuracy: 0.7756 - loss: 0.6956 - val_accuracy: 0.8233 - val_loss: 0.5845
Epoch 7/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 623s 1s/step - accuracy: 0.7895 - loss: 0.6529 - val_accuracy: 0.8146 - val_loss: 0.6016
Epoch 8/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 617s 1s/step - accuracy: 0.7771 - loss: 0.685

In [8]:
# -----------------------------
# Save model
# ----------------------------- 
model.save("small_data_10_epochs.h5")
print("Model saved as asl_mobilenetv2.h5")

Model saved as asl_mobilenetv2.h5


In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# -----------------------------
# Load model
# -----------------------------
model = load_model("small_data_10_epochs.h5")
print("Model loaded!")

# -----------------------------
# Unfreeze the last 30 layers of the model
# -----------------------------
for layer in model.layers[-30:]:
    layer.trainable = True

# Verify
trainable_count = np.sum([layer.trainable for layer in model.layers])
print(f"Trainable layers in whole model: {trainable_count}")

# -----------------------------
# Recompile with smaller LR
# -----------------------------
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Early stopping
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# -----------------------------
# Fine-tune
# -----------------------------
with tf.device("/GPU:0"):
    history_stage2 = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=10,
        callbacks=[early_stop],
        batch_size=64
    )
    

c:\Users\moham\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Model loaded!
Trainable layers in whole model: 31
Epoch 1/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 669s 1s/step - accuracy: 0.6417 - loss: 1.3312 - val_accuracy: 0.8560 - val_loss: 0.4785
Epoch 2/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 615s 1s/step - accuracy: 0.8273 - loss: 0.5384 - val_accuracy: 0.8938 - val_loss: 0.3331
Epoch 3/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 527s 867ms/step - accuracy: 0.8738 - loss: 0.3920 - val_accuracy: 0.9121 - val_loss: 0.2681
Epoch 4/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 600s 987ms/step - accuracy: 0.8972 - loss: 0.3076 - val_accuracy: 0.9259 - val_loss: 0.2221
Epoch 5/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 642s 1s/step - accuracy: 0.9108 - loss: 0.2613 - val_accuracy: 0.9362 - val_loss: 0.1937
Epoch 6/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 588s 968ms/step - accuracy: 0.9334 - loss: 0.2008 - val_accuracy: 0.9426 - val_loss: 0.1745
Epoch 7/10
608/608 ━━━━━━━━━━━━━━━━━━━━ 541s 890ms/step - accuracy: 0.9386 - loss: 0.1847 - val_accuracy: 0.9519 - val_loss: 0.1484
Epoch 8/10
608/608 ━━━━━━━━━━━━━━━━

In [8]:
# Save fine-tuned model
model.save("fine_tuned_asl_mobilenetv2.h5")
print("Fine-tuned model saved!")


Fine-tuned model saved!
